In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict ['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields = ["section","text","question"],
    keyword_fields = ['course']
)

In [7]:
q = 'the course has already started. can i still enrol?'

In [8]:
index.fit(documents)

In [18]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict= {'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [19]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
import os
from huggingface_hub import InferenceClient

In [12]:
client = InferenceClient()

In [13]:
q

'the course has already started. can i still enrol?'

In [15]:
response

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content="Yes, you may still be able to enroll in a course even after it has started, but this depends on the institution's policies and the course structure. Here’s what to do:\n\n1. **Check Deadlines**: Some institutions allow late enrollment with certain deadlines (e.g., within the first week or before key modules begin).  \n2. **Contact the Course Provider**: Reach out to the admissions office, program coordinator, or instructor to confirm if late enrollment is possible. They may also guide you on how to catch up on missed material.  \n\n3. **Online/In-Person Flexibility**: Some courses (especially online or self-paced ones) may permit late enrollment more easily than traditional classroom settings.  \n\n4. **Practical Considerations**: Confirm any prerequisites, fees, or required adjustments (e.g., accessing past lectures/materials).  \n\n*

In [24]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ question. 
Use only the facts from the CONTEXT when answering the question.
If the CONTEXT does not have the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [22]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [23]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [27]:
prompt = prompt_template.format(question =q, context=context).strip()

In [28]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ question. 
Use only the facts from the CONTEXT when answering the question.
If the CONTEXT does not have the answer, output NONE.

QUESTION: the course has already started. can i still enrol?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fina

In [29]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3-0324",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)


In [32]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."